<a href="https://colab.research.google.com/github/rajorlalala/GCAN/blob/master/%E4%BA%BA%E5%AE%B6%E7%9A%84GCAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# coding: utf-8

# In[ ]

#data_size: the total number of data you want to put in the model( for example: twitter15 is 742)
#retweet_user_size: the length of retweet propagation you want to utilize( in the paper we use 40 retweet users)
data_size=742##推特15
retweet_user_size=40

##read the Data 
import numpy as np
import os
path=r"/content/drive/MyDrive/GCAN/data/rumor_detection_acl2017/twitter15/user_feature_10"###感觉是类似tree形式的每个推文的转发用户的特征,顺序按照传播顺序来排列
files = os.listdir(path)#返回该路径下的文件名称列表
a=[]
k=[]
for file in files:#改成in files？
    f = np.load(r"/content/drive/MyDrive/GCAN/data/rumor_detection_acl2017/twitter15/user_feature_10/{}".format(file))#.txt要不要删？
    results=[]
    #k.append(file)    
    #next(f)#读取f中的下一行，第一次使用next()时读取的是第一行，删？
    for line in f:#因为next(f)所以是从第二行开始循环
        results.append(list(map(float,line)))#results.append([父uid,父推文id,父时间,子uid,子推文id,子时间])每一行的
    results=np.array(results)
    results=results.tolist()
    a.append(results)  #一个result是一篇推文的数据
    
##let all the rewteet user size of news be the same让所有转发用户的个数相同
import random
from sklearn import preprocessing
data_all=[]
for i in range(0,data_size):
    if len(a[i])>=retweet_user_size:
        k=a[i][0:retweet_user_size]
        
        data_all.append(k)
    else:
        a[i]=np.asarray(a[i])
        q=a[i][np.random.choice(a[i].shape[0],retweet_user_size,replace=True),:]#np.random.choice()返回的是rewteet_user_size个【0~a[i].shape[0]即该推文的行数】范围内的数组成的列表
        q=q.tolist()
        a[i]=a[i].tolist()
        k=a[i].extend(q)
        k=a[i][0:retweet_user_size]#[[],[],[],....,[]]
        
        data_all.append(k)
        
##use the user profile to calculate their cosine similarity for building the graph使用用户简介计算cos相似度
import networkx as nx
from sklearn.metrics.pairwise import cosine_similarity
#path=r"  "###删？
#files= os.listdir(path)#删？
a=[]
cos=[]
for data in data_all:#改成 for data in data_all:
    adj=[]
    #data=pd.read_csv(data_all[j])#删？
    
    for i in range(0,retweet_user_size):
        a=data[i]
        a=np.array(a)
        a=a.reshape(1,-1)#数据排成一行，维数不变
        similar=[]
        
        for k in range(0,retweet_user_size):
            b=data[k]
            b=np.array(b)
            b=b.reshape(1,-1)
            cosine=cosine_similarity(a,b)
            similar.append(np.round(cosine,2))#一个用户与其他用户的相似度
            
        similar=np.array(similar)
        similar=similar.flatten()
        adj.append(similar)#40个用户的similar组成adj
    cos.append(adj)#742个adj组成cos
    #[[[u1与其他u],[],...,[]],[[],[],...,[]],...,[[],[],...,[]]]

 
#encode the news content
import pandas as pd
import numpy as np
with open(r"/content/drive/MyDrive/GCAN/data/rumor_detection_acl2017/twitter15/new_TF_source_tweets.txt", 'r') as f: # read all the news content ###填源推文的路径(在此之前我们要先把真假新闻挑出来,id+content)
    #next(f)#删
    contents = f.readlines()

#vocab_size: how many different words in the news content  
from keras.preprocessing.text import one_hot
vocab_size=30###填
encoded_docs=[one_hot(d,30) for d in contents]#改成for d.split('\t')[1] in contents

from keras.preprocessing.sequence import pad_sequences
padded_docs=pad_sequences(encoded_docs,maxlen=30 ,padding="post") #let all the word embedding be the same length  ###填maxlen


================================================================================

In [ ]:
# coding: utf-8

##=====for here, we define some model functions to be utilized at the Training.py============##


##graph convolution network
import scipy.sparse as sp
def normalize_adj(adj, symmetric=True):
    if symmetric:
        d = sp.diags(np.power(np.array(adj.sum(1)), -0.5).flatten(), 0)#度矩阵的-0.5次幂
        a_norm = adj.dot(d).transpose().dot(d).tocsr()#拉普拉斯矩阵
    else:
        d = sp.diags(np.power(np.array(adj.sum(1)), -1).flatten(), 0)
        a_norm = d.dot(adj).tocsr()
    return a_norm


def normalize_adj_numpy(adj, symmetric=True):
    if symmetric:
        d = np.diag(np.power(np.array(adj.sum(1)), -0.5).flatten(), 0)
        a_norm = adj.dot(d).transpose().dot(d)
    else:
        d = np.diag(np.power(np.array(adj.sum(1)), -1).flatten(), 0)
        a_norm = d.dot(adj)
    return a_norm


def preprocess_adj(adj, symmetric=True):
    adj = adj + sp.eye(adj.shape[0])
    adj = normalize_adj(adj, symmetric)
    return adj


def preprocess_adj_numpy(adj, symmetric=True):
    adj = adj + np.eye(adj.shape[0])
    adj = normalize_adj_numpy(adj, symmetric)
    return adj


def preprocess_adj_tensor(adj_tensor, symmetric=True):
    adj_out_tensor = []
    for i in range(adj_tensor.shape[0]):
        adj = adj_tensor[i]
        adj = adj + np.eye(adj.shape[0])#???感觉应该删掉
        adj = normalize_adj_numpy(adj, symmetric)
        adj_out_tensor.append(adj)
    adj_out_tensor = np.array(adj_out_tensor)
    return adj_out_tensor#所有文章的拉普拉斯矩阵

def preprocess_adj_tensor_with_identity(adj_tensor, symmetric=True):
    adj_out_tensor = []
    for i in range(adj_tensor.shape[0]):
        adj = adj_tensor[i]
        adj = adj + np.eye(adj.shape[0])
        adj = normalize_adj_numpy(adj, symmetric)
        adj = np.concatenate([np.eye(adj.shape[0]), adj], axis=0)
        adj_out_tensor.append(adj)
    adj_out_tensor = np.array(adj_out_tensor)
    return adj_out_tensor

import keras.backend as K
import tensorflow as tf


def graph_conv_op(x, num_filters, graph_conv_filters, kernel):

    if len(x.get_shape()) == 2:
        conv_op = K.dot(graph_conv_filters, x)
        conv_op = tf.split(conv_op, num_filters, axis=0)
        conv_op = K.concatenate(conv_op, axis=1)
    elif len(x.get_shape()) == 3:
        conv_op = K.batch_dot(graph_conv_filters, x)
        conv_op = tf.split(conv_op, num_filters, axis=1)
        conv_op = K.concatenate(conv_op, axis=2)
    else:
        raise ValueError('x must be either 2 or 3 dimension tensor'
                         'Got input shape: ' + str(x.get_shape()))

    conv_out = K.dot(conv_op, kernel)
    return conv_out

from keras import activations, initializers, constraints
from keras import regularizers
import keras.backend as K
from keras.layers import Layer
import tensorflow as tf



class MultiGraphCNN(Layer):

    def __init__(self,
                 output_dim,
                 num_filters,
                 activation=None,
                 use_bias=True,
                 kernel_initializer='glorot_uniform',
                 bias_initializer='zeros',
                 kernel_regularizer=None,
                 bias_regularizer=None,
                 activity_regularizer=None,
                 kernel_constraint=None,
                 bias_constraint=None,
                 **kwargs):
        super(MultiGraphCNN, self).__init__(**kwargs)

        self.output_dim = output_dim
        self.num_filters = num_filters

        self.activation = activations.get(activation)
        self.use_bias = use_bias
        self.kernel_initializer = initializers.get(kernel_initializer)
        self.kernel_initializer.__name__ = kernel_initializer
        self.bias_initializer = initializers.get(bias_initializer)
        self.kernel_regularizer = regularizers.get(kernel_regularizer)
        self.bias_regularizer = regularizers.get(bias_regularizer)
        self.activity_regularizer = regularizers.get(activity_regularizer)
        self.kernel_constraint = constraints.get(kernel_constraint)
        self.bias_constraint = constraints.get(bias_constraint)

    def build(self, input_shape):

        if self.num_filters != int(input_shape[1][-2]/input_shape[1][-1]):
            raise ValueError('num_filters does not match with graph_conv_filters dimensions.')

        self.input_dim = input_shape[0][-1]
        kernel_shape = (self.num_filters * self.input_dim, self.output_dim)

        self.kernel = self.add_weight(shape=kernel_shape,
                                      initializer=self.kernel_initializer,
                                      name='kernel',
                                      regularizer=self.kernel_regularizer,
                                      constraint=self.kernel_constraint)
        if self.use_bias:
            self.bias = self.add_weight(shape=(self.output_dim,),
                                        initializer=self.bias_initializer,
                                        name='bias',
                                        regularizer=self.bias_regularizer,
                                        constraint=self.bias_constraint)
        else:
            self.bias = None

        self.built = True

    def call(self, inputs):

        output = graph_conv_op(inputs[0], self.num_filters, inputs[1], self.kernel)
        if self.use_bias:
            output = K.bias_add(output, self.bias)
        if self.activation is not None:
            output = self.activation(output)
        return output

    def compute_output_shape(self, input_shape):
        output_shape = (input_shape[0][0], input_shape[0][1], self.output_dim)
        return output_shape

    def get_config(self):
        config = {
            'output_dim': self.output_dim,
            'num_filters': self.num_filters,
            'activation': activations.serialize(self.activation),
            'use_bias': self.use_bias,
            'kernel_initializer': initializers.serialize(self.kernel_initializer),
            'bias_initializer': initializers.serialize(self.bias_initializer),
            'kernel_regularizer': regularizers.serialize(self.kernel_regularizer),
            'bias_regularizer': regularizers.serialize(self.bias_regularizer),
            'activity_regularizer': regularizers.serialize(self.activity_regularizer),
            'kernel_constraint': constraints.serialize(self.kernel_constraint),
            'bias_constraint': constraints.serialize(self.bias_constraint)
        }
        base_config = super(MultiGraphCNN, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

    

source_tweet_output_dim=32
source_tweet_length=30
number_of_feature=10#文章4.1
cnn_output_dim=32
cnn_output_length=38
filter_size=3
co_attention_output_dim=64
output_dim=32

##Dual Co-Attention: source tweet and graph-aware interaction embeddings
from keras import backend as k
from keras.layers import Layer,Permute

class coattention(Layer):

    def __init__(self, output_dim, **kwargs):
        self.output_dim = output_dim
        super(coattention, self).__init__(**kwargs)

    def build(self, input_shape):
        
        
        self.kernelW = self.add_weight(name='Wall', 
                                      shape=(source_tweet_output_dim, GCN_output_dim),
                                      initializer='uniform',
                                      trainable=True)
        self.kernelWs = self.add_weight(name='Ws', 
                                      shape=(retweet_user_size,retweet_user_size),
                                      initializer='uniform',
                                      trainable=True)
        self.kernelWc = self.add_weight(name='Wc', 
                                      shape=(source_tweet_length,source_tweet_length),
                                      initializer='uniform',
                                      trainable=True)
        self.kernelas = self.add_weight(name='Was', 
                                      shape=(GCN_output_dim,1),
                                      initializer='uniform',
                                      trainable=True)
        self.kernelac = self.add_weight(name='Wac', 
                                      shape=(source_tweet_output_dim,1),
                                      initializer='uniform',
                                      trainable=True)
        super(coattention, self).build(input_shape)  


    def call(self, x):
        C=x[0]#即wembed，源推文嵌入S
       
        print("C.shape",C.shape)
        RNN=Permute((2,1))(x[1])#G的转置G'
        
        f=k.dot(C,self.kernelW)#SW
        print("f.shape",f.shape)
        F=k.tanh(k.batch_dot(f,RNN))#F'=tanh(SWG')
        print("F.shape",F.shape)
        
        s=k.dot(RNN,self.kernelWs)#G'W
        print("s.shape",s.shape)
        c=k.dot(Permute((2,1))(C),self.kernelWc)#S'W
        print("c.shape",c.shape)
       
        Hs=k.tanh(s+k.batch_dot(c,F))#tanh(G'W+S'WF')
        print("Hs.shape",Hs.shape)
        Hc=k.tanh(c+k.batch_dot(s,Permute((2,1))(F)))#tanh(S'W+G'WF')
        print("Hc.shape",Hc.shape)
        
        
        As=k.softmax(k.dot(Permute((2,1))(Hs),self.kernelas))
        print("As.shape",As.shape)
        Ac=k.softmax(k.dot(Permute((2,1))(Hc),self.kernelac))
        print("Ac.shape",Ac.shape)
        
        As=Permute((2,1))(As)
        print("As.shape",As.shape)
        Ac=Permute((2,1))(Ac)
        print("Ac.shape",Ac.shape)
        
        sfinal=k.batch_dot(As,Permute((2,1))(RNN))
        
        print("sfinal.shape",sfinal.shape)
        cfinal=k.batch_dot(Ac,C)
       
        print("cfinal.shape",cfinal.shape)
        
        return keras.layers.concatenate([sfinal,cfinal])

    def compute_output_shape(self, input_shape):
        return (None, 64)
    
##Dual Co-Attention: source tweet and user propagation embeddings
from keras import backend as k
from keras.layers import Layer

class cocnnattention(Layer):#cocnnattention跟上面的coattention就只有def build中的shape不一样

    def __init__(self, output_dim, **kwargs):
        self.output_dim = output_dim
        super(cocnnattention, self).__init__(**kwargs)

    def build(self, input_shape):
        
        
        self.kernelW = self.add_weight(name='Wall', 
                                      shape=(source_tweet_output_dim, cnn_output_dim),
                                      initializer='uniform',
                                      trainable=True)
        self.kernelWs = self.add_weight(name='Ws', 
                                      shape=(cnn_output_length, cnn_output_length),
                                      initializer='uniform',
                                      trainable=True)
        self.kernelWc = self.add_weight(name='Wc', 
                                      shape=(source_tweet_length, source_tweet_length),
                                      initializer='uniform',
                                      trainable=True)
        self.kernelas = self.add_weight(name='Was', 
                                      shape=(cnn_output_dim,1),
                                      initializer='uniform',
                                      trainable=True)
        self.kernelac = self.add_weight(name='Wac', 
                                      shape=(source_tweet_output_dim,1),
                                      initializer='uniform',
                                      trainable=True)
        super(cocnnattention, self).build(input_shape)  


    def call(self, x):
        C=x[0]
       
        print("C.shape",C.shape)
        RNN=Permute((2,1))(x[1])
        
        f=k.dot(C,self.kernelW)
        print("f.shape",f.shape)
        F=k.tanh(k.batch_dot(f,RNN))
        print("F.shape",F.shape)
        
        s=k.dot(RNN,self.kernelWs)
        print("s.shape",s.shape)
        c=k.dot(Permute((2,1))(C),self.kernelWc)
        print("c.shape",c.shape)
       
        Hs=k.tanh(s+k.batch_dot(c,F))
        print("Hs.shape",Hs.shape)
        Hc=k.tanh(c+k.batch_dot(s,Permute((2,1))(F)))
        print("Hc.shape",Hc.shape)
        
        
        As=k.softmax(k.dot(Permute((2,1))(Hs),self.kernelas))
        print("As.shape",As.shape)
        Ac=k.softmax(k.dot(Permute((2,1))(Hc),self.kernelac))
        print("Ac.shape",Ac.shape)
        
        As=Permute((2,1))(As)
        Ac=Permute((2,1))(Ac)
        
        sfinal=k.batch_dot(As,Permute((2,1))(RNN))
        
        print("sfinal.shape",sfinal.shape)
        cfinal=k.batch_dot(Ac,C)
       
        print("cfinal.shape",cfinal.shape)
        
        return keras.layers.concatenate([sfinal,cfinal])

    def compute_output_shape(self, input_shape):
        return (None, 64)


================================================================

In [ ]:
y = []
with open(r'/content/drive/MyDrive/GCAN/data/rumor_detection_acl2017/twitter15/TF_label.txt') as f:
  for i in f.readlines():
    #print(i)
    if i == 'true'+'\n' or i == 'true':
      y.append(0)
    if i == 'false'+'\n' or i == 'false':
      y.append(1)################
len(y)

742

In [ ]:
import keras
keras.__version__

'2.7.0'

In [ ]:

# coding: utf-8

# In[ ]:




#==================================================================================================#
#evaluation
def f1_score(y_true, y_pred):
    #calculate tp、tn、fp、fn
    tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=0)
    tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=0)
    fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)
	
    
    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())
    
    f1 = 2*p*r / (p+r+K.epsilon())
    f1 = tf.where(tf.math.is_nan(f1), tf.zeros_like(f1), f1)
    return f1

def precision(y_true, y_pred):
    #calculate tp、tn、fp、fn
    tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=0)
    tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=0)
    fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)
	
   
    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())
    #calculate f1
    f1 = 2*p*r / (p+r+K.epsilon())
    f1 = tf.where(tf.math.is_nan(f1), tf.zeros_like(f1), f1)
    return p

def recall(y_true, y_pred):
    #calculate tp、tn、fp、fn
    tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=0)
    tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=0)
    fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)
	
    
    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())
    #calculate f1
    f1 = 2*p*r / (p+r+K.epsilon())
    f1 = tf.where(tf.math.is_nan(f1), tf.zeros_like(f1), f1)
    return r
#==============================================================================================================#
##==Start to build and train GCAN model (need to run data_preparation.py and GCAN.py first.)=======##

#data preparation
matrix=np.array(cos)  #cos is calculated from data_preparation.py, use the user profile to calculate their cosine similarity for building the graph.
graph_conv_filters=preprocess_adj_tensor(matrix)
#y=pd.read_csv(r".csv")###填标签的路径
#y=y[0:data_size]

#打乱数据集
#import random
#SEED = 1
#random.seed(SEED)
#random.shuffle(data_all)
#random.seed(SEED)
#random.shuffle(padded_docs)
#random.seed(SEED)
#random.shuffle(graph_conv_filters)
#random.seed(SEED)
#random.shuffle(y)

from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(data_all,y,test_size=0.3,random_state=0 )#用户特征  ###填随机种子
WX_train,WX_test,y_train,y_test=train_test_split(padded_docs,y,test_size=0.3,random_state=0 )#源推文独热编码
MX_train,MX_test,y_train,y_test=train_test_split(graph_conv_filters,y,test_size=0.3,random_state=0)#拉普拉斯矩阵


#下面这个cnnX_train要改成知识word2vec    
cnnX_train=np.reshape(X_train,(len(X_train),retweet_user_size,number_of_feature,1))#training_size改为X_train.shape[0]，number_of_feature=10
#know_X_train,know_X_test,y_train,y_test=train_test_split(data_know,y,test_size=0.3,random_state= )#data_know：对于每篇推文，先求每个实体的w2v，再求平均，最后(742*300)
#cnnknow_X_train=np.reshape(know_X_train,(训练集大小,30,10,1))
cnnX_test=np.reshape(X_test,(len(X_test),retweet_user_size,number_of_feature,1))#同上

from keras.utils.np_utils import to_categorical
y_train= to_categorical(y_train,2)#变成train_number行，2列的0，1矩阵，1：[0,1]；0：[1，0]
y_train= y_train.astype('int')
y_test= to_categorical(y_test,2)
y_test= y_test.astype('int')
    
num_filters = 1
source_tweet_output_dim=32
source_tweet_length=30
number_of_feature=10
cnn_output_dim=32
cnn_output_length=38#38改为28？
filter_size=3
co_attention_output_dim=64
output_dim=64#我把32改成了64
GCN_output_dim=32#论文第六页
retweet_user_size=40

from keras.layers import Input,Embedding,Reshape,GRU,AveragePooling1D,Flatten,Conv2D,Dense,Dropout
from keras import Model
winput=Input(shape=(source_tweet_length,)) #source_tweet_length:in the paper is 30
wembed=Embedding(vocab_size,source_tweet_output_dim,input_length=source_tweet_length)(winput)
wembed=Reshape((source_tweet_length,source_tweet_output_dim))(wembed)  #source_tweet_output_dim: define by yourself
wembed=GRU(source_tweet_output_dim,return_sequences=True)(wembed)#(None, 30, 32)

#user propagation representation左4
rmain_input =Input(shape=(retweet_user_size,number_of_feature)) #number_of_feature: in the paper is 10, retweet_user_size: in the paper is 40
rnnencoder=GRU(output_dim,return_sequences=True)(rmain_input)
rnnoutput1= AveragePooling1D(retweet_user_size)(rnnencoder)
rnnoutput=Flatten()(rnnoutput1)

#Graph-aware Propagation Representation左1
graph_conv_filters_input = Input(shape=(retweet_user_size, retweet_user_size))
gmain_input= MultiGraphCNN(GCN_output_dim, num_filters)([rmain_input, graph_conv_filters_input])
gmain_input= MultiGraphCNN(GCN_output_dim, num_filters)([gmain_input, graph_conv_filters_input])

#dual co attention
gco=coattention(co_attention_output_dim)([wembed,gmain_input])
gco=Flatten()(gco)
    
cmain_input=Input(shape=(retweet_user_size,number_of_feature,1))#左3
#shape=(30,10,1)
cnnco=Conv2D(cnn_output_dim,kernel_size=(filter_size,number_of_feature),activation="sigmoid")(cmain_input)#(None, 38, 1, 32) 
#删
maxpooling=Reshape((cnn_output_length,cnn_output_dim))(cnnco)#(None, 38, 32) 
#cmain_input=Reshape((28,32))(cmain_input)
co=cocnnattention(co_attention_output_dim)([wembed,maxpooling])
#maxpooling改为cmain_input
co=Flatten()(co)

merged_vector=keras.layers.concatenate([co,gco,rnnoutput])
x=Dense(output_dim,activation="relu")(merged_vector)
Dropout(0.1)
x=Dense(output_dim,activation="relu")(x)
Dropout(0.1)
x=Dense(output_dim,activation="relu")(x)
Dropout(0.1)
prediction=Dense(2,activation="softmax")(x)
#prediction=Dense(1,activation="sigmoid")(x) #Here is the final output:class{0,1}###我把2改成了1，softmax改成了sigmoid
    
model=Model([winput,rmain_input,cmain_input, graph_conv_filters_input],prediction)
model.summary()

from keras import optimizers
optimizer=tf.keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)##############
#optimizer=tf.keras.optimizers.SGD(lr=0.000001)
model.compile(optimizer=optimizer,loss="categorical_crossentropy",metrics=['accuracy', f1_score,precision,recall])###我把categorical_crossentropy改成了binary_crossentropy

#from keras.callbacks import EarlyStopping
#early_stopping = EarlyStopping(monitor='val_loss', patience=5, verbose=2)
history=model.fit([np.array(WX_train),np.array(X_train),np.array(cnnX_train),np.array(MX_train)],np.array(y_train),batch_size=128,epochs=1000,validation_split=0.1)#, callbacks=[early_stopping])###########
scores=model.evaluate([np.array(WX_test),np.array(X_test),np.array(cnnX_test),np.array(MX_test)],np.array(y_test), verbose=0)
#cnnX_train都改成know_X_train; cnnX_test都改成know_X_test


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in power


C.shape (None, 30, 32)
f.shape (None, 30, 32)
F.shape (None, 30, 40)
s.shape (None, 32, 40)
c.shape (None, 32, 30)
Hs.shape (None, 32, 40)
Hc.shape (None, 32, 30)
As.shape (None, 40, 1)
Ac.shape (None, 30, 1)
As.shape (None, 1, 40)
Ac.shape (None, 1, 30)
sfinal.shape (None, 1, 32)
cfinal.shape (None, 1, 32)
C.shape (None, 30, 32)
f.shape (None, 30, 32)
F.shape (None, 30, 38)
s.shape (None, 32, 38)
c.shape (None, 32, 30)
Hs.shape (None, 32, 38)
Hc.shape (None, 32, 30)
As.shape (None, 38, 1)
Ac.shape (None, 30, 1)
sfinal.shape (None, 1, 32)
cfinal.shape (None, 1, 32)
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 30)]         0           []                               
                                                                                                  
 embedding (Embed

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


Epoch 1/1000
C.shape (None, 30, 32)
f.shape (None, 30, 32)
F.shape (None, 30, 40)
s.shape (None, 32, 40)
c.shape (None, 32, 30)
Hs.shape (None, 32, 40)
Hc.shape (None, 32, 30)
As.shape (None, 40, 1)
Ac.shape (None, 30, 1)
As.shape (None, 1, 40)
Ac.shape (None, 1, 30)
sfinal.shape (None, 1, 32)
cfinal.shape (None, 1, 32)
C.shape (None, 30, 32)
f.shape (None, 30, 32)
F.shape (None, 30, 38)
s.shape (None, 32, 38)
c.shape (None, 32, 30)
Hs.shape (None, 32, 38)
Hc.shape (None, 32, 30)
As.shape (None, 38, 1)
Ac.shape (None, 30, 1)
sfinal.shape (None, 1, 32)
cfinal.shape (None, 1, 32)
C.shape (None, 30, 32)
f.shape (None, 30, 32)
F.shape (None, 30, 40)
s.shape (None, 32, 40)
c.shape (None, 32, 30)
Hs.shape (None, 32, 40)
Hc.shape (None, 32, 30)
As.shape (None, 40, 1)
Ac.shape (None, 30, 1)
As.shape (None, 1, 40)
Ac.shape (None, 1, 30)
sfinal.shape (None, 1, 32)
cfinal.shape (None, 1, 32)
C.shape (None, 30, 32)
f.shape (None, 30, 32)
F.shape (None, 30, 38)
s.shape (None, 32, 38)
c.shape (None,

In [ ]:
scores

[3.138198137283325,
 0.5022421479225159,
 0.4970991313457489,
 0.4985051155090332,
 0.49851104617118835]

In [ ]:
print(model.metrics_names)

['loss', 'accuracy', 'f1_score', 'precision', 'recall']


In [ ]:
y_pred = model.predict([np.array(WX_test),np.array(X_test),np.array(cnnX_test),np.array(MX_test)])
print(y_pred)

In [ ]:
pip uninstall keras

Found existing installation: Keras 2.2.4
Uninstalling Keras-2.2.4:
  Would remove:
    /usr/local/lib/python3.7/dist-packages/Keras-2.2.4.dist-info/*
    /usr/local/lib/python3.7/dist-packages/docs/*
    /usr/local/lib/python3.7/dist-packages/keras/*
  Would not remove (might be manually added):
    /usr/local/lib/python3.7/dist-packages/docs/md_autogen.py
    /usr/local/lib/python3.7/dist-packages/docs/update_docs.py
Proceed (y/n)? y
  Successfully uninstalled Keras-2.2.4


In [ ]:
pip install keras==2.2.4 -i http://pypi.douban.com/simple --trusted-host pypi.douban.com

Looking in indexes: http://pypi.douban.com/simple


=========================================================================================

In [ ]:
import zipfile


zFile = zipfile.ZipFile("GCAN.zip", "r")
    #ZipFile.namelist(): 获取ZIP文档内所有文件的名称列表
for fileM in zFile.namelist(): 
    zFile.extract(fileM, "")
zFile.close()

In [ ]:
conda install networkx
conda install numpy==1.14.3
conda install scikit-learn==0.21
conda install pandas==0.23.0

In [ ]:
# coding: utf-8

# In[ ]

#data_size: the total number of data you want to put in the model( for example: twitter15 is 742)
#retweet_user_size: the length of retweet propagation you want to utilize( in the paper we use 40 retweet users)
data_size=742##推特15
retweet_user_size=40

##read the Data 
import numpy as np
import os
path=r"GCAN/data/rumor_detection_acl2017/twitter15/user_feature_10"###感觉是类似tree形式的每个推文的转发用户的特征,顺序按照传播顺序来排列
files = os.listdir(path)#返回该路径下的文件名称列表
a=[]
k=[]
for file in files:#改成in files？
    f = np.load(r"GCAN/data/rumor_detection_acl2017/twitter15/user_feature_10/{}".format(file))#.txt要不要删？
    results=[]
    #k.append(file)    
    #next(f)#读取f中的下一行，第一次使用next()时读取的是第一行，删？
    for line in f:#因为next(f)所以是从第二行开始循环
        results.append(list(map(float,line)))#results.append([父uid,父推文id,父时间,子uid,子推文id,子时间])每一行的
    results=np.array(results)
    results=results.tolist()
    a.append(results)  #一个result是一篇推文的数据
    
##let all the rewteet user size of news be the same让所有转发用户的个数相同
import random
from sklearn import preprocessing
data_all=[]
for i in range(0,data_size):
    if len(a[i])>=retweet_user_size:
        k=a[i][0:retweet_user_size]
        
        data_all.append(k)
    else:
        a[i]=np.asarray(a[i])
        q=a[i][np.random.choice(a[i].shape[0],retweet_user_size,replace=True),:]#np.random.choice()返回的是rewteet_user_size个【0~a[i].shape[0]即该推文的行数】范围内的数组成的列表
        q=q.tolist()
        a[i]=a[i].tolist()
        k=a[i].extend(q)
        k=a[i][0:retweet_user_size]#[[],[],[],....,[]]
        
        data_all.append(k)
        
##use the user profile to calculate their cosine similarity for building the graph使用用户简介计算cos相似度
import networkx as nx
from sklearn.metrics.pairwise import cosine_similarity
#path=r"  "###删？
#files= os.listdir(path)#删？
a=[]
cos=[]
for data in data_all:#改成 for data in data_all:
    adj=[]
    #data=pd.read_csv(data_all[j])#删？
    
    for i in range(0,retweet_user_size):
        a=data[i]
        a=np.array(a)
        a=a.reshape(1,-1)#数据排成一行，维数不变
        similar=[]
        
        for k in range(0,retweet_user_size):
            b=data[k]
            b=np.array(b)
            b=b.reshape(1,-1)
            cosine=cosine_similarity(a,b)
            similar.append(np.round(cosine,2))#一个用户与其他用户的相似度
            
        similar=np.array(similar)
        similar=similar.flatten()
        adj.append(similar)#40个用户的similar组成adj
    cos.append(adj)#742个adj组成cos
    #[[[u1与其他u],[],...,[]],[[],[],...,[]],...,[[],[],...,[]]]

 
#encode the news content
import pandas as pd
import numpy as np
with open(r"GCAN/data/rumor_detection_acl2017/twitter15/new_TF_source_tweets.txt", 'r') as f: # read all the news content ###填源推文的路径(在此之前我们要先把真假新闻挑出来,id+content)
    #next(f)#删
    contents = f.readlines()

#vocab_size: how many different words in the news content  
from keras.preprocessing.text import one_hot
vocab_size=30###填
encoded_docs=[one_hot(d,30) for d in contents]#改成for d.split('\t')[1] in contents

from keras.preprocessing.sequence import pad_sequences
padded_docs=pad_sequences(encoded_docs,maxlen=30 ,padding="post") #let all the word embedding be the same length  ###填maxlen


In [ ]:
# coding: utf-8

##=====for here, we define some model functions to be utilized at the Training.py============##


##graph convolution network
import scipy.sparse as sp
def normalize_adj(adj, symmetric=True):
    if symmetric:
        d = sp.diags(np.power(np.array(adj.sum(1)), -0.5).flatten(), 0)#度矩阵的-0.5次幂
        a_norm = adj.dot(d).transpose().dot(d).tocsr()#拉普拉斯矩阵
    else:
        d = sp.diags(np.power(np.array(adj.sum(1)), -1).flatten(), 0)
        a_norm = d.dot(adj).tocsr()
    return a_norm


def normalize_adj_numpy(adj, symmetric=True):
    if symmetric:
        d = np.diag(np.power(np.array(adj.sum(1)), -0.5).flatten(), 0)
        a_norm = adj.dot(d).transpose().dot(d)
    else:
        d = np.diag(np.power(np.array(adj.sum(1)), -1).flatten(), 0)
        a_norm = d.dot(adj)
    return a_norm


def preprocess_adj(adj, symmetric=True):
    adj = adj + sp.eye(adj.shape[0])
    adj = normalize_adj(adj, symmetric)
    return adj


def preprocess_adj_numpy(adj, symmetric=True):
    adj = adj + np.eye(adj.shape[0])
    adj = normalize_adj_numpy(adj, symmetric)
    return adj


def preprocess_adj_tensor(adj_tensor, symmetric=True):
    adj_out_tensor = []
    for i in range(adj_tensor.shape[0]):
        adj = adj_tensor[i]
        adj = adj + np.eye(adj.shape[0])#???感觉应该删掉
        adj = normalize_adj_numpy(adj, symmetric)
        adj_out_tensor.append(adj)
    adj_out_tensor = np.array(adj_out_tensor)
    return adj_out_tensor#所有文章的拉普拉斯矩阵

def preprocess_adj_tensor_with_identity(adj_tensor, symmetric=True):
    adj_out_tensor = []
    for i in range(adj_tensor.shape[0]):
        adj = adj_tensor[i]
        adj = adj + np.eye(adj.shape[0])
        adj = normalize_adj_numpy(adj, symmetric)
        adj = np.concatenate([np.eye(adj.shape[0]), adj], axis=0)
        adj_out_tensor.append(adj)
    adj_out_tensor = np.array(adj_out_tensor)
    return adj_out_tensor

import keras.backend as K
import tensorflow as tf


def graph_conv_op(x, num_filters, graph_conv_filters, kernel):

    if len(x.get_shape()) == 2:
        conv_op = K.dot(graph_conv_filters, x)
        conv_op = tf.split(conv_op, num_filters, axis=0)
        conv_op = K.concatenate(conv_op, axis=1)
    elif len(x.get_shape()) == 3:
        conv_op = K.batch_dot(graph_conv_filters, x)
        conv_op = tf.split(conv_op, num_filters, axis=1)
        conv_op = K.concatenate(conv_op, axis=2)
    else:
        raise ValueError('x must be either 2 or 3 dimension tensor'
                         'Got input shape: ' + str(x.get_shape()))

    conv_out = K.dot(conv_op, kernel)
    return conv_out

from keras import activations, initializers, constraints
from keras import regularizers
import keras.backend as K
from keras.layers import Layer
import tensorflow as tf



class MultiGraphCNN(Layer):

    def __init__(self,
                 output_dim,
                 num_filters,
                 activation=None,
                 use_bias=True,
                 kernel_initializer='glorot_uniform',
                 bias_initializer='zeros',
                 kernel_regularizer=None,
                 bias_regularizer=None,
                 activity_regularizer=None,
                 kernel_constraint=None,
                 bias_constraint=None,
                 **kwargs):
        super(MultiGraphCNN, self).__init__(**kwargs)

        self.output_dim = output_dim
        self.num_filters = num_filters

        self.activation = activations.get(activation)
        self.use_bias = use_bias
        self.kernel_initializer = initializers.get(kernel_initializer)
        self.kernel_initializer.__name__ = kernel_initializer
        self.bias_initializer = initializers.get(bias_initializer)
        self.kernel_regularizer = regularizers.get(kernel_regularizer)
        self.bias_regularizer = regularizers.get(bias_regularizer)
        self.activity_regularizer = regularizers.get(activity_regularizer)
        self.kernel_constraint = constraints.get(kernel_constraint)
        self.bias_constraint = constraints.get(bias_constraint)

    def build(self, input_shape):

        if self.num_filters != int(input_shape[1][-2]/input_shape[1][-1]):
            raise ValueError('num_filters does not match with graph_conv_filters dimensions.')

        self.input_dim = input_shape[0][-1]
        kernel_shape = (self.num_filters * self.input_dim, self.output_dim)

        self.kernel = self.add_weight(shape=kernel_shape,
                                      initializer=self.kernel_initializer,
                                      name='kernel',
                                      regularizer=self.kernel_regularizer,
                                      constraint=self.kernel_constraint)
        if self.use_bias:
            self.bias = self.add_weight(shape=(self.output_dim,),
                                        initializer=self.bias_initializer,
                                        name='bias',
                                        regularizer=self.bias_regularizer,
                                        constraint=self.bias_constraint)
        else:
            self.bias = None

        self.built = True

    def call(self, inputs):

        output = graph_conv_op(inputs[0], self.num_filters, inputs[1], self.kernel)
        if self.use_bias:
            output = K.bias_add(output, self.bias)
        if self.activation is not None:
            output = self.activation(output)
        return output

    def compute_output_shape(self, input_shape):
        output_shape = (input_shape[0][0], input_shape[0][1], self.output_dim)
        return output_shape

    def get_config(self):
        config = {
            'output_dim': self.output_dim,
            'num_filters': self.num_filters,
            'activation': activations.serialize(self.activation),
            'use_bias': self.use_bias,
            'kernel_initializer': initializers.serialize(self.kernel_initializer),
            'bias_initializer': initializers.serialize(self.bias_initializer),
            'kernel_regularizer': regularizers.serialize(self.kernel_regularizer),
            'bias_regularizer': regularizers.serialize(self.bias_regularizer),
            'activity_regularizer': regularizers.serialize(self.activity_regularizer),
            'kernel_constraint': constraints.serialize(self.kernel_constraint),
            'bias_constraint': constraints.serialize(self.bias_constraint)
        }
        base_config = super(MultiGraphCNN, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

    

source_tweet_output_dim=32
source_tweet_length=30
number_of_feature=10#文章4.1
cnn_output_dim=32
cnn_output_length=38
filter_size=3
co_attention_output_dim=64
output_dim=32

##Dual Co-Attention: source tweet and graph-aware interaction embeddings
from keras import backend as k
from keras.layers import Layer,Permute

class coattention(Layer):

    def __init__(self, output_dim, **kwargs):
        self.output_dim = output_dim
        super(coattention, self).__init__(**kwargs)

    def build(self, input_shape):
        
        
        self.kernelW = self.add_weight(name='Wall', 
                                      shape=(source_tweet_output_dim, GCN_output_dim),
                                      initializer='uniform',
                                      trainable=True)
        self.kernelWs = self.add_weight(name='Ws', 
                                      shape=(retweet_user_size,retweet_user_size),
                                      initializer='uniform',
                                      trainable=True)
        self.kernelWc = self.add_weight(name='Wc', 
                                      shape=(source_tweet_length,source_tweet_length),
                                      initializer='uniform',
                                      trainable=True)
        self.kernelas = self.add_weight(name='Was', 
                                      shape=(GCN_output_dim,1),
                                      initializer='uniform',
                                      trainable=True)
        self.kernelac = self.add_weight(name='Wac', 
                                      shape=(source_tweet_output_dim,1),
                                      initializer='uniform',
                                      trainable=True)
        super(coattention, self).build(input_shape)  


    def call(self, x):
        C=x[0]#即wembed，源推文嵌入S
       
        print("C.shape",C.shape)
        RNN=Permute((2,1))(x[1])#G的转置G'
        
        f=k.dot(C,self.kernelW)#SW
        print("f.shape",f.shape)
        F=k.tanh(k.batch_dot(f,RNN))#F'=tanh(SWG')
        print("F.shape",F.shape)
        
        s=k.dot(RNN,self.kernelWs)#G'W
        print("s.shape",s.shape)
        c=k.dot(Permute((2,1))(C),self.kernelWc)#S'W
        print("c.shape",c.shape)
       
        Hs=k.tanh(s+k.batch_dot(c,F))#tanh(G'W+S'WF')
        print("Hs.shape",Hs.shape)
        Hc=k.tanh(c+k.batch_dot(s,Permute((2,1))(F)))#tanh(S'W+G'WF')
        print("Hc.shape",Hc.shape)
        
        
        As=k.softmax(k.dot(Permute((2,1))(Hs),self.kernelas))
        print("As.shape",As.shape)
        Ac=k.softmax(k.dot(Permute((2,1))(Hc),self.kernelac))
        print("Ac.shape",Ac.shape)
        
        As=Permute((2,1))(As)
        print("As.shape",As.shape)
        Ac=Permute((2,1))(Ac)
        print("Ac.shape",Ac.shape)
        
        sfinal=k.batch_dot(As,Permute((2,1))(RNN))
        
        print("sfinal.shape",sfinal.shape)
        cfinal=k.batch_dot(Ac,C)
       
        print("cfinal.shape",cfinal.shape)
        
        return keras.layers.concatenate([sfinal,cfinal])

    def compute_output_shape(self, input_shape):
        return (None, 64)
    
##Dual Co-Attention: source tweet and user propagation embeddings
from keras import backend as k
from keras.layers import Layer

class cocnnattention(Layer):#cocnnattention跟上面的coattention就只有def build中的shape不一样

    def __init__(self, output_dim, **kwargs):
        self.output_dim = output_dim
        super(cocnnattention, self).__init__(**kwargs)

    def build(self, input_shape):
        
        
        self.kernelW = self.add_weight(name='Wall', 
                                      shape=(source_tweet_output_dim, cnn_output_dim),
                                      initializer='uniform',
                                      trainable=True)
        self.kernelWs = self.add_weight(name='Ws', 
                                      shape=(cnn_output_length, cnn_output_length),
                                      initializer='uniform',
                                      trainable=True)
        self.kernelWc = self.add_weight(name='Wc', 
                                      shape=(source_tweet_length, source_tweet_length),
                                      initializer='uniform',
                                      trainable=True)
        self.kernelas = self.add_weight(name='Was', 
                                      shape=(cnn_output_dim,1),
                                      initializer='uniform',
                                      trainable=True)
        self.kernelac = self.add_weight(name='Wac', 
                                      shape=(source_tweet_output_dim,1),
                                      initializer='uniform',
                                      trainable=True)
        super(cocnnattention, self).build(input_shape)  


    def call(self, x):
        C=x[0]
       
        print("C.shape",C.shape)
        RNN=Permute((2,1))(x[1])
        
        f=k.dot(C,self.kernelW)
        print("f.shape",f.shape)
        F=k.tanh(k.batch_dot(f,RNN))
        print("F.shape",F.shape)
        
        s=k.dot(RNN,self.kernelWs)
        print("s.shape",s.shape)
        c=k.dot(Permute((2,1))(C),self.kernelWc)
        print("c.shape",c.shape)
       
        Hs=k.tanh(s+k.batch_dot(c,F))
        print("Hs.shape",Hs.shape)
        Hc=k.tanh(c+k.batch_dot(s,Permute((2,1))(F)))
        print("Hc.shape",Hc.shape)
        
        
        As=k.softmax(k.dot(Permute((2,1))(Hs),self.kernelas))
        print("As.shape",As.shape)
        Ac=k.softmax(k.dot(Permute((2,1))(Hc),self.kernelac))
        print("Ac.shape",Ac.shape)
        
        As=Permute((2,1))(As)
        Ac=Permute((2,1))(Ac)
        
        sfinal=k.batch_dot(As,Permute((2,1))(RNN))
        
        print("sfinal.shape",sfinal.shape)
        cfinal=k.batch_dot(Ac,C)
       
        print("cfinal.shape",cfinal.shape)
        
        return keras.layers.concatenate([sfinal,cfinal])

    def compute_output_shape(self, input_shape):
        return (None, 64)

In [ ]:
y = []
with open(r'GCAN/data/rumor_detection_acl2017/twitter15/TF_label.txt') as f:
  for i in f.readlines():
    #print(i)
    if i == 'true'+'\n' or i == 'true':
      y.append(0)
    if i == 'false'+'\n' or i == 'false':
      y.append(1)################
len(y)

In [ ]:

# coding: utf-8

# In[ ]:




#==================================================================================================#
#evaluation
def f1_score(y_true, y_pred):
    #calculate tp、tn、fp、fn
    tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=0)
    tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=0)
    fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)
	
    
    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())
    
    f1 = 2*p*r / (p+r+K.epsilon())
    f1 = tf.where(tf.is_nan(f1), tf.zeros_like(f1), f1)###要不要把tf.is_nan改为tf.math.is_nan？
    return f1

def precision(y_true, y_pred):
    #calculate tp、tn、fp、fn
    tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=0)
    tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=0)
    fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)
	
   
    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())
    #calculate f1
    f1 = 2*p*r / (p+r+K.epsilon())
    f1 = tf.where(tf.is_nan(f1), tf.zeros_like(f1), f1)###要不要把tf.is_nan改为tf.math.is_nan？
    return p

def recall(y_true, y_pred):
    #calculate tp、tn、fp、fn
    tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=0)
    tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=0)
    fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)
	
    
    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())
    #calculate f1
    f1 = 2*p*r / (p+r+K.epsilon())
    f1 = tf.where(tf.is_nan(f1), tf.zeros_like(f1), f1)###要不要把tf.is_nan改为tf.math.is_nan？
    return r
#==============================================================================================================#
##==Start to build and train GCAN model (need to run data_preparation.py and GCAN.py first.)=======##

#data preparation
matrix=np.array(cos)   #cos is calculated from data_preparation.py, use the user profile to calculate their cosine similarity for building the graph.
graph_conv_filters=preprocess_adj_tensor(matrix)
#y=pd.read_csv(r".csv")###填标签的路径
#y=y[0:data_size]

from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(data_all,y,test_size=0.3,random_state=0 )#用户特征  ###填随机种子
WX_train,WX_test,y_train,y_test=train_test_split(padded_docs,y,test_size=0.3,random_state=0 )#源推文独热编码
MX_train,MX_test,y_train,y_test=train_test_split(graph_conv_filters,y,test_size=0.3,random_state=0)#拉普拉斯矩阵
#下面这个cnnX_train要改成知识word2vec    
cnnX_train=np.reshape(X_train,(len(X_train),retweet_user_size,number_of_feature,1))#training_size改为X_train.shape[0]，number_of_feature=10
#know_X_train,know_X_test,y_train,y_test=train_test_split(data_know,y,test_size=0.3,random_state= )#data_know：对于每篇推文，先求每个实体的w2v，再求平均，最后(742*300)
#cnnknow_X_train=np.reshape(know_X_train,(训练集大小,30,10,1))
cnnX_test=np.reshape(X_test,(len(X_test),retweet_user_size,number_of_feature,1))#同上

print('--------------------------',WX_train.shape)

from keras.utils.np_utils import to_categorical
y_train= to_categorical(y_train,2)#变成train_number行，2列的0，1矩阵，1：[0,1]；0：[1，0]
y_train= y_train.astype('int')
y_test= to_categorical(y_test,2)
y_test= y_test.astype('int')
    
num_filters = 1
source_tweet_output_dim=32
source_tweet_length=30
number_of_feature=10
cnn_output_dim=32
cnn_output_length=38
filter_size=3
co_attention_output_dim=64
output_dim=32
GCN_output_dim=32#论文第六页
retweet_user_size=40

from keras.layers import Input,Embedding,Reshape,GRU,AveragePooling1D,Flatten,Conv2D,Dense,Dropout
from keras import Model
#source tweet encoding左2
winput=Input(shape=(source_tweet_length,)) #source_tweet_length:in the paper is 30
print('--------------------------',winput.shape)
wembed=Embedding(vocab_size,source_tweet_output_dim,input_length=source_tweet_length)(winput)
wembed=Reshape((source_tweet_length,source_tweet_output_dim))(wembed)  #source_tweet_output_dim: define by yourself
wembed=GRU(source_tweet_output_dim,return_sequences=True)(wembed)#(None, 30, 32)

#print('--------------------------',wembed.shape)

#user propagation representation左4
rmain_input =Input(shape=(retweet_user_size,number_of_feature)) #number_of_feature: in the paper is 10, retweet_user_size: in the paper is 40
rnnencoder=GRU(output_dim,return_sequences=True)(rmain_input)
rnnoutput1= AveragePooling1D(retweet_user_size)(rnnencoder)
rnnoutput=Flatten()(rnnoutput1)

#Graph-aware Propagation Representation左1
graph_conv_filters_input = Input(shape=(retweet_user_size, retweet_user_size))
gmain_input= MultiGraphCNN(GCN_output_dim, num_filters)([rmain_input, graph_conv_filters_input])
gmain_input= MultiGraphCNN(GCN_output_dim, num_filters)([gmain_input, graph_conv_filters_input])

#dual co attention
gco=coattention(co_attention_output_dim)([wembed,gmain_input])
gco=Flatten()(gco)
    
cmain_input=Input(shape=(retweet_user_size,number_of_feature,1))#左3
#shape=(30,10,1)
cnnco=Conv2D(cnn_output_dim,kernel_size=(filter_size,number_of_feature),activation="sigmoid")(cmain_input)#(None, 38, 1, 32) 
#删
maxpooling=Reshape((cnn_output_length,cnn_output_dim))(cnnco)#(None, 38, 32) 
#cmain_input=Reshape((28,32))(cmain_input)
co=cocnnattention(co_attention_output_dim)([wembed,maxpooling])
#maxpooling改为cmain_input
co=Flatten()(co)

merged_vector=keras.layers.concatenate([co,gco,rnnoutput])
x=Dense(output_dim,activation="relu")(merged_vector)
x=Dense(output_dim,activation="relu")(x)
x=Dense(output_dim,activation="relu")(x)
prediction=Dense(2,activation="softmax")(x) #Here is the final output:class{0,1}
    
model=Model([winput,rmain_input,cmain_input, graph_conv_filters_input],prediction)
model.summary()
  
from keras import optimizers
Adam=keras.optimizers.Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
model.compile(optimizer=Adam,loss="categorical_crossentropy",metrics=['accuracy', f1_score,precision,recall])
from keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=5, verbose=2)
history=model.fit([np.array(WX_train),np.array(X_train),np.array(cnnX_train),np.array(MX_train)],np.array(y_train),epochs=50,validation_split=0.1, callbacks=[early_stopping])
scores=model.evaluate([np.array(WX_test),np.array(X_test),np.array(cnnX_test),np.array(MX_test)],np.array(y_test), verbose=0)
#cnnX_train都改成know_X_train; cnnX_test都改成know_X_test


In [ ]:
import keras
keras.__version__

import numpy
numpy.__version__